# QUESTÃO DE NEGÓCIO

- Foi nos requisitado por um CFO de uma rede de farmácias, a previsão de vendas de todas as lojas pelas proximas 6 semanas, onde ele deseja saber quando deve ser investido em cada uma delas para realizar uma reforma nas lojas.

# 0.0 Imports

In [1]:
import pandas as pd
import numpy as np
import inflection
import math
import seaborn as sns

from sklearn.preprocessing import RobustScaler, MinMaxScaler
from scipy import stats as ss
from matplotlib import pyplot as plt
from IPython.core.display import HTML
from IPython.display import Image
from sklearn.preprocessing import RobustScaler, MinMaxScaler, LabelEncoder

## 0.1 Helper Fuctions

In [2]:
def jupyter_settings(): 
    %matplotlib inline 
    %pylab inline
    
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    
    display( HTML( '<style>.container { width:100% !important; }</style>') ) 
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )
    
    sns.set()
    
def cramer_v( x, y ):
    cm = pd.crosstab( x, y ).as_matrix() 
    n = cm.sum()
    r, k = cm.shape
    
    chi2 = ss.chi2_contingency( cm )[0]
    chi2corr = max( 0, chi2 - (k-1)*(r-1)/(n-1) )
    
    kcorr = k - (k-1)**2/(n-1)
    rcorr = r - (r-1)**2/(n-1)
    
    return np.sqrt( (chi2corr/n) / ( min( kcorr-1, rcorr-1 ) ) )


In [3]:
jupyter_settings()

## 0.2 Loading Data

In [4]:
df_sales_raw = pd.read_csv('data/train.csv', low_memory=False)
df_store_raw = pd.read_csv('data/store.csv', low_memory=False)

# merge
df_raw = pd.merge(df_sales_raw, df_store_raw, how='left', on='Store')

In [5]:
df_raw.head()

# 1.0 Descrição dos Dados

In [6]:
df = df_raw.copy()

## 1.1 Rename Columns

In [7]:
df.columns

In [8]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo','StateHoliday', 
            'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth',
            'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']

snakecase = lambda x: inflection.underscore(x)
cols_new = list(map(snakecase, cols_old))

df.columns = cols_new

## 1.2 Data Dimensions

In [9]:
print('Number of Rows: {}'.format(df.shape[0]))
print('Number of Cols: {}'.format(df.shape[1]))

## 1.3 Data Types

In [10]:
# object to datetime 'date' column
df['date'] = pd.to_datetime(df['date'])
df.dtypes

## 1.4 Check NA's

In [11]:
# competition_distance
df['competition_distance'] = df['competition_distance'].apply (lambda x: 200000.0 if math.isnan(x) else x)

# competition_open_since_month
df['competition_open_since_month'] = df.apply (lambda x: x['date'].month if math.isnan(x['competition_open_since_month']) else x['competition_open_since_month'], axis=1)

# competition_open_since_year
df['competition_open_since_year'] = df.apply (lambda x: x['date'].year if math.isnan(x['competition_open_since_year']) else x['competition_open_since_year'], axis=1)

# promo2_since_week
df['promo2_since_week'] = df.apply (lambda x: x['date'].week if math.isnan(x['promo2_since_week']) else x['promo2_since_week'], axis=1)

# promo2_since_year
df['promo2_since_year'] = df.apply (lambda x: x['date'].year if math.isnan(x['promo2_since_year']) else x['promo2_since_year'], axis=1)

# promo interval
month_map = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sept', 10: 'Oct', 11: 'Nov', 12: 'Dec'}

df['promo_interval'].fillna(0, inplace=True)

df['month_map'] = df['date'].dt.month.map(month_map)
df['is_promo'] = df[['promo_interval', 'month_map']].apply (lambda x: 0 if x['promo_interval'] == 0 else 1 if x['month_map'] in x['promo_interval'].split (',') else 0, axis=1)

## 1.6 Change types

In [12]:
df['competition_open_since_month'] = df['competition_open_since_month'].astype(int)
df['competition_open_since_year'] = df['competition_open_since_year'].astype(int)
df['competition_distance'] = df['competition_distance'].astype(int)

df['promo2_since_week'] = df['promo2_since_week'].astype(int)
df['promo2_since_year'] = df['promo2_since_year'].astype(int)

## 1.7 Descriptive Statistical

In [13]:
# numerical attributes
num_attributes = df.select_dtypes (include=['int64', 'float64'])

# categorical attributes
cat_attributes = df.select_dtypes (exclude=['int64', 'float64', 'datetime64[ns]'])

### 1.7.1 Numerical attributes

In [14]:
# central tendency - mean, median
ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

# dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame(num_attributes.apply(np.std)).T
d2 = pd.DataFrame(num_attributes.apply(np.min)).T
d3 = pd.DataFrame(num_attributes.apply(np.max)).T
d4 = pd.DataFrame(num_attributes.apply( lambda x: x.max() - x.min())).T
d5 = pd.DataFrame(num_attributes.apply( lambda x: x.skew())).T
d6 = pd.DataFrame(num_attributes.apply( lambda x: x.kurtosis())).T

# concat
m = pd.concat([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
m.columns = ['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis']

In [15]:
sns.displot(df['competition_distance'], kde=False)

### 1.7.2 Categorical attributes

In [16]:
cat_attributes.apply(lambda x: x.unique().shape[0])

In [17]:
aux = df[(df['state_holiday'] !='0') & (df['sales'] > 0)]

plt.subplot(1, 3, 1)
sns.boxplot (x='state_holiday', y='sales', data=aux)

plt.subplot(1, 3, 2)
sns.boxplot (x='store_type', y='sales', data=aux)

plt.subplot(1, 3, 3)
sns.boxplot (x='assortment', y='sales', data=aux)

# 2.0 Feature Engineering

In [18]:
df2 = df.copy()

## 2.1 Mapa mental de hipoteses

In [19]:
Image('img/mind_map.png')

## 2.2 Lista final de hipoteses

1. Lojas com maior sortimentos deveriam vender mais.
2. Lojas com competidores mais próximos deveriam vender menos.
3. Lojas com competidores à mais tempo deveriam vendem mais.
4. Lojas com promoções ativas por mais tempo deveriam vender mais.
5. Lojas com mais dias de promoção deveriam vender mais.
7. Lojas com mais promoções consecutivas deveriam vender mais. 
8. Lojas abertas durante o feriado de Natal deveriam vender mais. 
9. Lojas deveriam vender mais ao longo dos anos.
10. Lojas deveriam vender mais no segundo semestre do ano.
11. Lojas deveriam vender mais depois do dia 10 de cada mês.
12. Lojas deveriam vender menos aos finais de semana.
13. Lojas deveriam vender menos durante os feriados escolares.

## 2.3 Feature Engineering

In [20]:
# year 
df2['year'] = df2['date'].dt.year

# month
df2['month'] = df2['date'].dt.month

# day 
df2['day'] = df2['date'].dt.day

# week of year
df2['week_of_year'] = df2['date'].dt.isocalendar().week

# year week
df2['year_week'] = df2['date'].dt.strftime('%Y-%W-')

# competition since
df2['competition_since'] = df2.apply(lambda x: datetime.datetime(year= x['competition_open_since_year'], month= x['competition_open_since_month'],day=1), axis=1)
df2['competition_time_month'] = (( df2['date'] - df2['competition_since'])/30).apply(lambda x: x.days).astype( int )

# promo since
df2['promo_since'] = df2['promo2_since_year'].astype( str ) + '-' + df2['promo2_since_week'].astype( str )
df2['promo_since'] = df2['promo_since'].apply( lambda x: datetime.datetime.strptime(x + '-1', '%Y-%W-%w') - datetime.timedelta(days=7))
df2['promo_time_week'] = ((df2['date'] - df2['promo_since'])/7).apply(lambda x: x.days).astype( int )

# assortment
df2['assortment'] = df2['assortment'].apply( lambda x: 'basic' if x == 'a' else 'extra' if x == 'b' else 'extended' )

# state holiday
df2['state_holiday'] = df2['state_holiday'].apply( lambda x: 'public_holiday' if x == 'a' else 'easter_holiday' if x == 'b' else 'christmas' if x == 'c' else 'regular_day')

# 3.0 Filtragem das Variáveis

In [21]:
df3 = df2.copy()

## 3.1 Filtragem das linhas

In [22]:
df3 = df3[(df3['open'] != 0) & (df3['sales'] > 0)]

## 3.2 Seleção das colunas

In [23]:
cols_drop = ['customers', 'open', 'promo_interval', 'month_map']
df3 = df3.drop( cols_drop, axis=1 )

# 4.0 Análise Exploratória

In [24]:
df4 = df3.copy()

## 4.1 Analise Univariada

### 4.1.1 Response Variable

In [25]:
sns.displot( df4['sales'], kde=False )

### 4.1.2 Numerical Variable

In [26]:
num_attributes.hist(bins=25);

### 4.1.3 Categorical Variable

In [27]:
# Converta as colunas para o tipo 'category'

df4['state_holiday'] = df4['state_holiday'].astype('category')
df4['store_type'] = df4['store_type'].astype('category')
df4['assortment'] = df4['assortment'].astype('category')

# state_holiday
plt.subplot(3, 2, 1)
sns.countplot(data=df4, x='state_holiday', hue='state_holiday')

plt.subplot(3, 2, 2)
for holiday in df4['state_holiday'].unique():
    subset = df4[df4['state_holiday'] == holiday]['sales']
    sns.kdeplot(subset, label=holiday, fill=True)
    
plt.xlabel('sales')
plt.ylabel('Density')

# store_type
plt.subplot(3, 2, 3)
sns.countplot(data=df4, x='store_type')

plt.subplot(3, 2, 4)
for store_type in df4['store_type'].unique():
    subset = df4[df4['store_type'] == store_type]['sales']
    sns.kdeplot(subset, label=store_type, fill=True)

plt.xlabel('sales')
plt.ylabel('Density')

# assortment
plt.subplot(3, 2, 5)
sns.countplot(data=df4, x='assortment')

plt.subplot(3, 2, 6)
for assortment in df4['assortment'].unique():
    subset = df4[df4['assortment'] == assortment]['sales']
    sns.kdeplot(subset, label=assortment, fill=True)

plt.xlabel('sales')
plt.ylabel('Density')

plt.tight_layout()
plt.show()

## 4.2 Analise Bivariada

### H1. Lojas com maior sortimentos deveriam vender mais.

**FALSO** Lojas com maior sortimento vendem menos.

In [28]:
aux1 = df4[['assortment', 'sales']].groupby('assortment').sum().reset_index()
sns.barplot(x= 'assortment', y= 'sales', data=aux1 );

aux2 = df4[['year_week','assortment', 'sales']].groupby(['year_week', 'assortment']).sum().reset_index()
aux2.pivot(index= 'year_week', columns= 'assortment', values= 'sales').plot()

aux3 = aux2[aux2['assortment'] == 'extended']
aux3.pivot(index= 'year_week', columns= 'assortment', values= 'sales').plot()

### H2. Lojas com competidores mais próximos deveriam vender menos.
- **FALSA** Lojas com competidores mais proximos vendem mais.

In [29]:
aux1 = df4[['competition_distance', 'sales']].groupby( 'competition_distance' ).sum().reset_index()

plt.subplot( 1, 3, 1 )
sns.scatterplot( x ='competition_distance', y='sales', data=aux1 );

plt.subplot( 1, 3, 2 )
bins = list( np.arange( 0, 20000, 1000) )
aux1['competition_distance_binned'] = pd.cut( aux1['competition_distance'], bins=bins )

aux2 = aux1[['competition_distance_binned', 'sales']].groupby('competition_distance_binned' ).sum().reset_index()
sns.barplot( x='competition_distance_binned', y='sales', data=aux2 );
plt.xticks( rotation=90 );

### H3. Lojas com competidores à mais tempo deveriam vendem mais.
- **FALSA** Lojas com COMPETIDORES À MAIS TEMPO vendem MENOS.

In [30]:
plt.subplot(1, 3, 1)
aux1 = df4[['competition_time_month', 'sales']].groupby('competition_time_month').sum().reset_index()

aux2 = aux1[( aux1['competition_time_month'] < 120 ) & (aux1['competition_time_month'] != 0 )]
sns.barplot( x='competition_time_month', y='sales', data=aux2 ); 
plt.xticks( rotation=90 );

plt.subplot( 1, 3, 2 )
sns.regplot( x='competition_time_month', y='sales', data=aux2 );

plt.subplot( 1, 3, 3 )
x = sns.heatmap( aux1.corr( method='pearson'), annot=True );

### H4. Lojas com promoções ativas por mais tempo deveriam vender mais.
- **FALSA** Lojas com promoções ativas por mais tempo vendem menos, depois de um certo periodo de promoção.

In [31]:
aux1 = df4[['promo_time_week', 'sales']].groupby( 'promo_time_week').sum().reset_index()

grid = GridSpec( 2, 3 )

plt.subplot( grid[0,0] )
aux2 = aux1[aux1['promo_time_week'] > 0] # promo extendido 
sns.barplot( x='promo_time_week', y='sales', data=aux2 ); 
plt.xticks( rotation=90 );

plt.subplot( grid[0,1] )
sns.regplot( x='promo_time_week', y='sales', data=aux2 );

plt.subplot( grid[1,0] )
aux3 = aux1[aux1['promo_time_week'] < 0] # promo regular 
sns.barplot( x='promo_time_week', y='sales', data=aux3 ); 
plt.xticks( rotation=90 );

plt.subplot( grid[1,1] )
sns.regplot( x='promo_time_week', y='sales', data=aux3 );

plt.subplot( grid[:,2] )
sns.heatmap( aux1.corr( method='pearson' ), annot=True );

### H5. Lojas com mais promoções consecutivas deveriam vender mais.
- **FALSA** Lojas com mais promoções consecutivas vendem menos

In [32]:
df4[['promo', 'promo2', 'sales']].groupby( ['promo', 'promo2'] ).sum().reset_index()

In [33]:
aux1 = df4[( df4['promo'] == 1 ) & ( df4['promo2'] == 1 )][['year_week','sales']].groupby( 'year_week' ).sum().reset_index()
ax = aux1.plot()

aux2 = df4[( df4['promo'] == 1 ) & ( df4['promo2'] == 0 )][['year_week','sales']].groupby( 'year_week' ).sum().reset_index()
aux2.plot( ax=ax )

ax.legend( labels=['Tradicional & Extendida', 'Extendida']);

### H6. Lojas abertas durante o feriado de Natal deveriam vender mais.
- **FALSA** Lojas abertas durante o feriado do Natal vendem menos.

In [34]:
aux = df4[df4['state_holiday'] != 'regular_day']

plt.subplot( 1, 2, 1 )
aux1 = aux[['state_holiday', 'sales']].groupby( 'state_holiday' ).sum().reset_index()
sns.barplot( x='state_holiday', y='sales', data=aux1 );

plt.subplot( 1, 2, 2 )
aux2 = aux[['year', 'state_holiday', 'sales']].groupby(['year', 'state_holiday']).sum().reset_index()
sns.barplot( x='year', y='sales', hue='state_holiday', data=aux2 );

### H7. Lojas deveriam vender mais ao longo dos anos.
- **FALSA** lojas vendem menos ao longo dos anos.

In [35]:
aux1 = df4[['year', 'sales']].groupby( 'year' ).sum().reset_index()

plt.subplot( 1, 3, 1 )
sns.barplot( x='year', y='sales', data=aux1 );

plt.subplot( 1, 3, 2 )
sns.regplot( x='year', y='sales', data=aux1 );

plt.subplot( 1, 3, 3 )
sns.heatmap( aux1.corr( method='pearson' ), annot=True );

### H8. Lojas deveriam vender mais no segundo semestre do ano.
- **FALSA** Lojas vendem menos no segundo semestre do ano

In [36]:
aux1 = df4[['month', 'sales']].groupby( 'month' ).sum().reset_index()

plt.subplot( 1, 3, 1 )
sns.barplot( x='month', y='sales', data=aux1 );

plt.subplot( 1, 3, 2 )
sns.regplot( x='month', y='sales', data=aux1 );

plt.subplot( 1, 3, 3 )
sns.heatmap( aux1.corr( method='pearson' ), annot=True );

### H9. Lojas deveriam vender mais depois do dia 10 de cada mês.
- **VERDADEIRA** Lojas vendem mais depois do dia 10 de cada mês.

In [37]:
aux1 = df4[['day', 'sales']].groupby( 'day' ).sum().reset_index()

plt.subplot( 2, 2, 1 )
sns.barplot( x='day', y='sales', data=aux1 );

plt.subplot( 2, 2, 2 )
sns.regplot( x='day', y='sales', data=aux1 );

plt.subplot( 2, 2, 3 )
sns.heatmap( aux1.corr( method='pearson' ), annot=True );
aux1['before_after'] = aux1['day'].apply( lambda x: 'before_10_days' if x <= 10 else 'after_10_days')
aux2 =aux1[['before_after', 'sales']].groupby( 'before_after' ).sum().reset_index()

plt.subplot( 2, 2, 4 )
sns.barplot( x='before_after', y='sales', data=aux2 );

### H10. Lojas deveriam vender menos aos fins de semana
**VERDADEIRA** Lojas vendem menos aos fins de semana

In [38]:
aux1 = df4[['day_of_week', 'sales']].groupby( 'day_of_week' ).sum().reset_index()

plt.subplot( 1, 3, 1 )
sns.barplot( x='day_of_week', y='sales', data=aux1 );

plt.subplot( 1, 3, 2 )
sns.regplot( x='day_of_week', y='sales', data=aux1 );

plt.subplot( 1, 3, 3 )
sns.heatmap( aux1.corr( method='pearson' ), annot=True );

### H11. Lojas deveriam vender menos durante os feriados escolares. 
- **VERDADEIRA** Lojas vendem menos durante os feriados escolares, exceto os meses de Julho e Agosto.

In [39]:
aux1 = df4[['school_holiday', 'sales']].groupby( 'school_holiday' ).sum().reset_index()

plt.subplot( 2, 1, 1 )
sns.barplot( x='school_holiday', y='sales', data=aux1 );

aux2 = df4[['month', 'school_holiday', 'sales']].groupby( ['month','school_holiday'] ).sum().reset_index()

plt.subplot( 2, 1, 2 )
sns.barplot( x='month', y='sales', hue='school_holiday', data=aux2 );

## 4.3 Analise Multivariada

### 4.3.1 Numerical Attributes

In [40]:
correlation = num_attributes.corr( method = 'pearson')
sns.heatmap(correlation, annot=True);

### 4.3.2 Categorical Attributes

In [41]:
# only categorical data
#a = df4.select_dtypes( include='object' )

# Calculate cramer V
#a1 = cramer_v( a['state_holiday'], a['state_holiday'] )
#a2 = cramer_v( a['state_holiday'], a['store_type'] )
#a3 = cramer_v( a['state_holiday'], a['assortment'] )
#a4 = cramer_v( a['store_type'], a['state_holiday'] )
#a5 = cramer_v( a['store_type'], a['store_type'] )
#a6 = cramer_v( a['store_type'], a['assortment'] )
#a7 = cramer_v( a['assortment'], a['state_holiday'] )
#a8 = cramer_v( a['assortment'], a['store_type'] )
#a9 = cramer_v( a['assortment'], a['assortment'] )

# Final dataset
#d = pd.DataFrame({'state_holiday': [a1, a2, a3], 'store_type': [a4, a5, a6], 'assortment': [a7, a8, a9]})

#d = d.set_index( d.columns )

#sns.heatmap( d, annot=True )

# 5.0 Preparação dos Dados

In [42]:
df5 = df4.copy()

## 5.1 Normalização

In [43]:
Image('img/normalizacao.png')

## 5.2 Rescaling

In [44]:
rs = RobustScaler()
mms = MinMaxScaler()

# competition distance
df5['competition_distance'] = rs.fit_transform( df5[['competition_distance']].values)

# competition time month
df5['competition_time_month'] = rs.fit_transform(df5[['competition_time_month']].values)

# promo time week
df5['promo_time_week'] = mms.fit_transform( df5[['promo_time_week']].values)

# year
df5['year'] = mms.fit_transform( df5[['year']].values)

## 5.3 Transformação

### 5.3.1 Encoding

In [45]:
# state holiday - One Hot Encoding
pd.get_dummies (df5, prefix=['state_holiday'], columns=['state_holiday'])

# store_type - Label Encoding
le = LabelEncoder()
df5['store_type'] = le.fit_transform(df5['store_type'])

# assortment - Ordinal Encoding
assortment_dict = {'basic': 1,  'extra': 2, 'extended': 3}
df5['assortment'] = df5['assortment'].map( assortment_dict )

### 5.3.2 Response Variable

In [46]:
df5['sales'] = np.log1p (df5['sales'])

### 5.3.3 Nature Transformation

In [47]:
# day of week
df5['day_of_week_sin'] = df5['day_of_week'].apply( lambda x: np.sin( x * ( 2. * np.pi/7 ) ) )
df5['day_of_week_cos'] = df5['day_of_week'].apply( lambda x: np.cos( x * ( 2. * np.pi/7 ) ) )

# month
df5['month_sin'] = df5['month'].apply( lambda x: np.sin( x * ( 2. * np.pi/12 ) ) )
df5['month_cos'] = df5['month'].apply( lambda x: np.cos( x * ( 2. * np.pi/12 ) ) )

# day
df5['day_sin'] = df5['day'].apply( lambda x: np.sin( x * ( 2. * np.pi/30 ) ) ) 
df5['day_cos'] = df5['day'].apply( lambda x: np.cos( x * ( 2. * np.pi/30 ) ) )

# week of year
df5['week_of_year_sin'] = df5['week_of_year'].apply( lambda x: np.sin( x * ( 2. * np.pi/52 ) ) )
df5['week_of_year_cos'] = df5['week_of_year'].apply( lambda x: np.cos( x * ( 2. * np.pi/52 ) ) )